In [1]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
header = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    "content-type":"text",
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [2]:
def get_html(link):
    time.sleep(1)
    req = requests.get(link, headers=header)
    return req.text

In [166]:
get_html('https://hotline.ua/mobile/mobilnye-telefony-i-smartfony/?p=0')[:4000]

'<!DOCTYPE html>\n<html>\n<head>\n        <script>\n    // Функция аналог jQuery(document).ready(function(){})\n    function gtm_documentReady(callback) {\n        if (document.readyState !== \'loading\' && document.readyState) {\n            callback();\n        } else {\n            document.addEventListener("DOMContentLoaded", callback);\n        }\n    };\n\n    // Функция установки куки\n    function gtm_setCookie(name, value, maxAge) {\n        var options = {\n            path: \'/\', secure: true, \'max-age\': maxAge\n        };\n\n        if (options.expires instanceof Date) {\n            options.expires = options.expires.toUTCString();\n        }\n\n        var updatedCookie = encodeURIComponent(name) + "=" + encodeURIComponent(value);\n\n        for (var optionKey in options) {\n            updatedCookie += "; " + optionKey;\n            var optionValue = options[optionKey];\n            if (optionValue !== true) {\n            updatedCookie += "=" + optionValue;\n         

In [167]:
def get_pages_amount(code):
    soup = BeautifulSoup(code)
    pages = soup.find('div', class_='pages-list cell-sm').find_all('a', class_='pages')[-1].get('href').split('=')[1]
    return pages

In [168]:
pages_amount = get_pages_amount(get_html('https://hotline.ua/mobile/mobilnye-telefony-i-smartfony/?p=1'))
pages_amount

'74'

In [169]:
base_link = 'https://hotline.ua/mobile/mobilnye-telefony-i-smartfony/?p='


In [170]:
def get_data(code, ind=0):
    soup = BeautifulSoup(code)
    
    phones = soup.find_all('li', class_='product-item')
    data = pd.DataFrame({'name': [], 'best_price': [], 'price_range': [], 'salers_amount': [], 'popularity': []})
    for phone in phones:
        try:
            name = phone.find('div', class_='item-info').find('p', class_='h4').find('a').text.replace('\n', '')
        except:
            name = None
            
        try:
            best_price = phone.find('div', class_='item-price stick-bottom').find('span', class_='value').text
        except:
            best_price = None
        
        try:
            price_range = phone.find('div', class_='item-price stick-bottom').find('div', class_='text-sm').text
        except:
            price_range = None
            
        try: 
            salers_amount = phone.find('div', class_='item-price stick-bottom').find('a', class_='link').text
        except:
            salers_amount = None
    
        try:
            stats = phone.find('div', class_='item-info').find('div', class_='text').find('p').text.strip()
        except: 
            stats = None
        t = pd.Series([name, best_price, price_range, salers_amount, stats])
        df_temp = pd.DataFrame({'name': t[0], 'best_price': t[1], 'price_range': t[2], 'salers_amount': t[3], 'popularity': t[4]}, index=range(ind, ind+1))
        ind += 1
        data = data.append(df_temp)
    return data, ind

In [272]:
result = pd.DataFrame({'name': [], 'best_price': [], 'price_range': [], 'salers_amount': [], 'popularity': []})
ind = 0
for i in tqdm(np.arange(75)):
    link = base_link + str(i)
    t, ind = get_data(get_html(link), ind)
    result = result.append(t)

100%|██████████| 75/75 [05:32<00:00,  4.44s/it]


In [150]:
copy_res = pd.read_csv('ds.csv')

In [151]:
result = copy_res

In [152]:
copy_res

,Unnamed: 0,brand_name,model_name,os,popularity,best_price,lowest_price,highest_price,sellers_amount,screen_size,memory_size,battery_size,release_date
0,0,ALCATEL,1 1/8GB Bluish Black (5033D-2JALUAA),Android,803,1690.0,1529.0,1819.0,36,5.00,8.0,2000.0,10.2020
1,1,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,902,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9.2020
2,2,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,926,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9.2020
3,3,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,938,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9.2020
4,4,Nokia,1.3 1/16GB Charcoal,Android,178,1999.0,NaN,NaN,10,5.71,16.0,3000.0,4.2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,1219,Apple,iPhone XS Max 64GB Gold (MT522),iOS,124,22685.0,16018.0,27900.0,61,6.50,64.0,3174.0,9.2018
1220,1220,Apple,iPhone XS Max Dual Sim 64GB Gold (MT732),iOS,695,24600.0,21939.0,33720.0,28,6.50,64.0,3174.0,9.2018
1221,1221,HUAWEI,nova 5T 6/128GB Black (51094MEU),Android,51,8804.0,7999.0,9999.0,18,6.26,128.0,3750.0,11.2019
1222,1222,ZTE,nubia Red Magic 5G 8/128GB Black,Android,473,18755.0,18500.0,19010.0,2,6.65,128.0,4500.0,10.2020


In [364]:
result.shape

(1275, 5)

In [365]:
result.shape

(1275, 5)

In [366]:
result.name = result.name.str.strip()

In [367]:
result.isna().sum()

name              51
best_price        51
price_range      311
salers_amount     51
popularity        51
dtype: int64

In [368]:
result = result.drop(result[result.name.isna()].index)

In [369]:
result.best_price = result.best_price.str.replace('\xa0', '')

In [370]:
result.best_price = result.best_price.astype('float')

In [371]:
brands = []
model_names = []
for brand in result.name:
    brands.append(brand[:brand.index(" ")])
    model_names.append(brand[brand.index(" ")+1:])

In [372]:
result['brand_name'] = brands
result['model_name'] = model_names

In [373]:
result.brand_name.unique()

array(['Apple', 'Google', 'Xiaomi', 'Samsung', 'OnePlus', 'realme',
       'Motorola', 'HUAWEI', 'Meizu', 'Nokia', 'OPPO', 'Sony', 'ASUS',
       'LG', 'Oukitel', 'CAT', 'ZTE', 'Tecno', 'DOOGEE', 'Honor',
       'Ulefone', 'Blackview', 'Sigma', 'vivo', 'Philips', 'Cubot',
       'Lenovo', 'Rezone', 'Nomi', 'BlackBerry', 'Fly', 'ERGO', 'UMIDIGI',
       'AGM', 'Vernee', 'ALCATEL', 'Maxcom', 'Land', 'HTC', '2E', 'NUU',
       'Sharp', 'Archos', 'Astro', 'Vodafone', 'Elephone', 'Microsoft',
       'Globex', 'S-TELL', 'Assistant', 'Jinga', 'Smartex', 'Bravis',
       'MAFAM', 'Viaan', 'Coolpad', 'General', 'Prestigio', 'Bluboo',
       'Crosscall', 'iOutdoor', 'LEAGOO', 'KENEKSI', 'myPhone'],
      dtype=object)

In [374]:
model_names[:3]

['iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3)',
 'Pixel 4a 6/128GB Just Black',
 'Poco X3 NFC 6/128GB Cobalt Blue']

In [375]:
result = result.drop(columns='name')

In [376]:
result

,best_price,price_range,salers_amount,popularity,brand_name,model_name
0,33992.0,31 849 – 42 999 грн,Всі пропозиції (101),"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...",Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3)
1,12106.0,11 740 – 12 999 грн,Всі пропозиції (37),"Смартфон • 1 SIM • екран: 5,81"" • OLED •...",Google,Pixel 4a 6/128GB Just Black
2,7152.0,6 836 – 7 999 грн,Всі пропозиції (59),Смартфон • 2 або 1 + карта пам'яті SIM • е...,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue
3,20889.0,18 200 – 26 499 грн,Всі пропозиції (93),"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...",Apple,iPhone 11 64GB Black (MWLT2)
4,9620.0,8 555 – 10 394 грн,Всі пропозиції (55),"Смартфон • 2 SIM • екран: 6,7"" • Super A...",Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD)
...,...,...,...,...,...,...
1270,326.0,222 – 373 грн,Всі пропозиції (14),"Мобільний телефон • 2 SIM • екран: 2,4"" • ...",Viaan,V241 (Black)
1271,5600.0,None,\n EuroMade\n,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...",myPhone,Hammer Energy Black
1272,2999.0,None,\n TTT.UA\n,Смартфон • 2 або 1 + карта пам'яті SIM • е...,Sigma,mobile X-treme PQ24 Black/Orange
1273,5628.0,5 297 – 6 584 грн,Всі пропозиції (11),Смартфон • 2 або 1 + карта пам'яті SIM • е...,AGM,A10 6/128GB Black


In [377]:
result.price_range = result.price_range.str.replace("грн", '')
result.price_range = result.price_range.str.replace('\xa0', '')
result

,best_price,price_range,salers_amount,popularity,brand_name,model_name
0,33992.0,31849–42999,Всі пропозиції (101),"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...",Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3)
1,12106.0,11740–12999,Всі пропозиції (37),"Смартфон • 1 SIM • екран: 5,81"" • OLED •...",Google,Pixel 4a 6/128GB Just Black
2,7152.0,6836–7999,Всі пропозиції (59),Смартфон • 2 або 1 + карта пам'яті SIM • е...,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue
3,20889.0,18200–26499,Всі пропозиції (93),"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...",Apple,iPhone 11 64GB Black (MWLT2)
4,9620.0,8555–10394,Всі пропозиції (55),"Смартфон • 2 SIM • екран: 6,7"" • Super A...",Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD)
...,...,...,...,...,...,...
1270,326.0,222–373,Всі пропозиції (14),"Мобільний телефон • 2 SIM • екран: 2,4"" • ...",Viaan,V241 (Black)
1271,5600.0,None,\n EuroMade\n,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...",myPhone,Hammer Energy Black
1272,2999.0,None,\n TTT.UA\n,Смартфон • 2 або 1 + карта пам'яті SIM • е...,Sigma,mobile X-treme PQ24 Black/Orange
1273,5628.0,5297–6584,Всі пропозиції (11),Смартфон • 2 або 1 + карта пам'яті SIM • е...,AGM,A10 6/128GB Black


In [378]:
result = result.fillna('none')

In [379]:
lowest_price = [] 
highest_price = []
for p_range in result.price_range:
    try:
        p_range.index('–')
        lowest_price.append(int(p_range.split('–')[0]))
        highest_price.append(int(p_range.split('–')[1]))
    except:
        lowest_price.append(None)
        highest_price.append(None)

In [380]:
result['lowest_price'] = lowest_price
result['highest_price'] = highest_price

In [381]:
len(lowest_price)

1224

In [382]:
result = result.drop(columns='price_range')

In [383]:
result

,best_price,salers_amount,popularity,brand_name,model_name,lowest_price,highest_price
0,33992.0,Всі пропозиції (101),"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...",Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0
1,12106.0,Всі пропозиції (37),"Смартфон • 1 SIM • екран: 5,81"" • OLED •...",Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0
2,7152.0,Всі пропозиції (59),Смартфон • 2 або 1 + карта пам'яті SIM • е...,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0
3,20889.0,Всі пропозиції (93),"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...",Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0
4,9620.0,Всі пропозиції (55),"Смартфон • 2 SIM • екран: 6,7"" • Super A...",Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0
...,...,...,...,...,...,...,...
1270,326.0,Всі пропозиції (14),"Мобільний телефон • 2 SIM • екран: 2,4"" • ...",Viaan,V241 (Black),222.0,373.0
1271,5600.0,\n EuroMade\n,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...",myPhone,Hammer Energy Black,NaN,NaN
1272,2999.0,\n TTT.UA\n,Смартфон • 2 або 1 + карта пам'яті SIM • е...,Sigma,mobile X-treme PQ24 Black/Orange,NaN,NaN
1273,5628.0,Всі пропозиції (11),Смартфон • 2 або 1 + карта пам'яті SIM • е...,AGM,A10 6/128GB Black,5297.0,6584.0


In [387]:
salers_am = []
for salers in result.salers_amount:
    try:
        salers.index('(')
        salers_am.append(salers[salers.index('(')+1:salers.index(')')])
    except:
        salers_am.append(1)

In [388]:
result.salers_amount = salers_am

In [389]:
result

,best_price,salers_amount,popularity,brand_name,model_name,lowest_price,highest_price
0,33992.0,101,"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...",Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0
1,12106.0,37,"Смартфон • 1 SIM • екран: 5,81"" • OLED •...",Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0
2,7152.0,59,Смартфон • 2 або 1 + карта пам'яті SIM • е...,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0
3,20889.0,93,"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...",Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0
4,9620.0,55,"Смартфон • 2 SIM • екран: 6,7"" • Super A...",Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0
...,...,...,...,...,...,...,...
1270,326.0,14,"Мобільний телефон • 2 SIM • екран: 2,4"" • ...",Viaan,V241 (Black),222.0,373.0
1271,5600.0,1,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...",myPhone,Hammer Energy Black,NaN,NaN
1272,2999.0,1,Смартфон • 2 або 1 + карта пам'яті SIM • е...,Sigma,mobile X-treme PQ24 Black/Orange,NaN,NaN
1273,5628.0,11,Смартфон • 2 або 1 + карта пам'яті SIM • е...,AGM,A10 6/128GB Black,5297.0,6584.0


In [390]:
result['stats'] = result.popularity

In [392]:
result = result.drop(columns='popularity')

In [409]:
sigma = result[result.brand_name == 'Sigma']
sigma.brand_name = 'Sigma mobile'
models = []
for name in sigma.model_name:
    models.append(name[7:])
sigma.model_name = models

/Users/artempozdniakov/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [411]:
result[result.brand_name == 'Sigma'] = sigma

In [412]:
result

,best_price,salers_amount,brand_name,model_name,lowest_price,highest_price,stats
0,33992.0,101,Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0,"Смартфон • 2 SIM • екран: 6,1"" • OLED • ..."
1,12106.0,37,Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0,"Смартфон • 1 SIM • екран: 5,81"" • OLED •..."
2,7152.0,59,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...
3,20889.0,93,Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0,"Смартфон • 2 SIM • екран: 6,1"" • IPS • ..."
4,9620.0,55,Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0,"Смартфон • 2 SIM • екран: 6,7"" • Super A..."
...,...,...,...,...,...,...,...
1270,326.0,14,Viaan,V241 (Black),222.0,373.0,"Мобільний телефон • 2 SIM • екран: 2,4"" • ..."
1271,5600.0,1,myPhone,Hammer Energy Black,NaN,NaN,"Смартфон • 2 SIM • екран: 5"" • IPS • 1..."
1272,2999.0,1,Sigma mobile,X-treme PQ24 Black/Orange,NaN,NaN,Смартфон • 2 або 1 + карта пам'яті SIM • е...
1273,5628.0,11,AGM,A10 6/128GB Black,5297.0,6584.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...


In [446]:
result.stats[100].split('•   ')

['Смартфон ',
 '2 SIM ',
 'екран: 6,5" ',
 'OLED ',
 '2688x1242 ',
 "вбудована пам'ять: 256 ГБ ",
 "оперативна пам'ять: 4 ГБ ",
 'процесор: Apple A12 Bionic ',
 'ОС: iOS ',
 'акумулятор: 3174 мАг (незнімний) ',
 'камера: 12 (f / 1.8) + 12 (f / 2.2) Мп ',
 'колір: золотий ',
 'NFC: + (Apple Pay)\n                        • 09.2018\n\n                                                    • Модель з лінійки  iPhone XS Max']

In [56]:
screen_size = []
memory_size = []
battery_size = []
release_date = []
system = []

for i in result.stats:
    if "екран:" in i:
        screen_size.append(i.split('екран:')[1][:5])
    else:
        screen_size.append(None)
    if "вбудована пам'ять:" in i:
        memory_size.append(i.split("вбудована пам'ять:")[1][:11])
    else:
        memory_size.append(None)
    if "акумулятор:" in i:
        battery_size.append(i.split("акумулятор:")[1][:22])
    else:
        battery_size.append(None)
    if ".20" in i:
        release_date.append(i.split(".20")[0][-3:]+".20"+i.split(".20")[1][:3])
    else:
        release_date.append(None)
    if "ОС:" in i:
        system.append(i.split("ОС:")[1][:11])
    else:
        system.append(None)
        
    
        
    
    


In [477]:
release_date[1001]

' 09.2016\n'

In [478]:
result.stats[101]

'Смартфон •   2 або 1 + карта пам\'яті SIM •   екран: 5,5" •   IPS •   1920x1080 •   вбудована пам\'ять: 64 ГБ •   оперативна пам\'ять: 4 ГБ •   процесор: Qualcomm Snapdragon 625 + GPU Adreno 506 •   ОС: Android 6.0 •   акумулятор: Li-Ion, 4100 мАч (незнімний) •   камера: 13 (F2.0) Мп •   колір: чорний\n                        • 05.2017\n\n                                                    • Модель з лінійки  Redmi Note 4x'

In [479]:
result.shape

(1224, 12)

In [480]:
result['screen_size'] = screen_size
result['memory_size'] = memory_size
result['battery_size'] = battery_size
result['release_date'] = release_date
result['system'] = system

In [481]:
result

,best_price,salers_amount,brand_name,model_name,lowest_price,highest_price,stats,screen_size,memory_size,battery_size,release_date,system
0,33992.0,101,Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0,"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...","6,1""",128 ГБ •,2815 мАг (незнімний),10.2020\n,Apple iOS
1,12106.0,37,Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0,"Смартфон • 1 SIM • екран: 5,81"" • OLED •...","5,81",128 ГБ •,"Li-Pol, 3140 мАг (нез",08.2020\n,Android 10
2,7152.0,59,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,"6,67",128 ГБ •,5160 мАг (незнімний),09.2020\n,Android 10
3,20889.0,93,Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0,"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...","6,1""",64 ГБ •,3110 мАг (незнімний),09.2019\n,iOS • ак
4,9620.0,55,Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0,"Смартфон • 2 SIM • екран: 6,7"" • Super A...","6,7""",128 ГБ •,7000 мАг (незнімний),09.2020\n,Android 10
...,...,...,...,...,...,...,...,...,...,...,...,...
1270,326.0,14,Viaan,V241 (Black),222.0,373.0,"Мобільний телефон • 2 SIM • екран: 2,4"" • ...","2,4""","0,032 ГБ •",1000 мАг (знімний) •,09.2016\n,None
1271,5600.0,1,myPhone,Hammer Energy Black,NaN,NaN,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...","5"" •",16 ГБ •,"Li-Pol, 5000 мАг (нез",01.2017,Android 6.
1272,2999.0,1,Sigma mobile,X-treme PQ24 Black/Orange,NaN,NaN,Смартфон • 2 або 1 + карта пам'яті SIM • е...,"5"" •",8 ГБ • о,Li-Pol 5000 mAh (незм,01.2017\n,Android 7.
1273,5628.0,11,AGM,A10 6/128GB Black,5297.0,6584.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,"5,7""",128 ГБ •,4400 мАг (незнімний),10.2020,Android 9.


In [47]:
result.to_csv('ds.csv')

In [48]:
result = pd.read_csv('ds.csv')

In [49]:
result['popularity'] = result['Unnamed: 0']

In [50]:
result = result.drop(columns='Unnamed: 0')

In [51]:
result.screen_size.unique()

array([6.1 , 5.81, 6.67, 6.7 , 6.53, 6.4 , 5.8 , 6.2 , 4.7 , 6.24, 6.5 ,
       5.4 , 6.49, 6.22, 6.26, 6.3 , 6.  , 6.8 , 5.7 , 6.78, 6.81, 6.55,
       6.9 , 6.59, 6.39, 5.5 , 5.99, 6.6 , 6.44, 7.6 , 5.  , 6.47, 6.57,
       2.4 , 6.58, 5.3 , 6.08, 5.71, 6.41,  nan, 6.15, 5.45, 6.43, 1.77,
       6.21, 6.56, 2.8 , 6.52, 3.5 , 6.82, 6.18, 5.9 , 6.51, 2.2 , 5.84,
       4.5 , 4.  , 5.6 , 6.34, 5.1 , 1.44, 1.8 , 7.3 , 6.88, 6.09, 5.65,
       5.2 , 3.2 , 6.65, 7.12, 5.95, 5.15, 6.35, 2.  , 6.19, 8.1 , 1.4 ,
       6.01, 4.93, 3.  , 2.3 , 5.93, 5.72])

In [13]:
result.screen_size = result.screen_size.str.replace('•','')
result.screen_size = result.screen_size.str.replace(' ','')
result.screen_size = result.screen_size.str.replace('"', '')
result.screen_size = result.screen_size.str.replace(',', '.')

array(['6.1', '5.81', '6.67', '6.7', '6.53', '6.4', '5.8', '6.2', '4.7',
       '6.24', '6.5', '5.4', '6.49', '6.22', '6.26', '6.3', '6', '6.8',
       '5.7', '6.78', '6.81', '6.55', '6.9', '6.59', '6.39', '5.5',
       '5.99', '6.6', '6.44', '7.6', '5', '6.47', '6.57', '2.4', '6.58',
       '5.3', '6.08', '5.71', '6.41', 'осно', '6.15', '5.45', '6.43',
       '1.77', '6.21', '6.56', '2.8', '6.52', '3.5', '6.82', '6.18',
       '5.9', '6.51', '2.2', '5.84', '4.5', '4', '5.6', '6.34', '5.1',
       '1.44', '1.8', '7.3', '6.88', '6.09', '5.65', '5.2', '3.2', '6.65',
       '8(в', '7.12', '5.95', '5.15', '6.35', '2', '6.19', '8.1', '1.4',
       '6.01', '4.93', '3', '2.3', '5.93', '5.72'], dtype=object)

In [16]:
cop = result[result.screen_size == 'осно']
cop.screen_size = None
result[result.screen_size == 'осно'] = cop

/Users/artempozdniakov/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [18]:
result.screen_size.unique()

array(['6.1', '5.81', '6.67', '6.7', '6.53', '6.4', '5.8', '6.2', '4.7',
       '6.24', '6.5', '5.4', '6.49', '6.22', '6.26', '6.3', '6', '6.8',
       '5.7', '6.78', '6.81', '6.55', '6.9', '6.59', '6.39', '5.5',
       '5.99', '6.6', '6.44', '7.6', '5', '6.47', '6.57', '2.4', '6.58',
       '5.3', '6.08', '5.71', '6.41', None, '6.15', '5.45', '6.43',
       '1.77', '6.21', '6.56', '2.8', '6.52', '3.5', '6.82', '6.18',
       '5.9', '6.51', '2.2', '5.84', '4.5', '4', '5.6', '6.34', '5.1',
       '1.44', '1.8', '7.3', '6.88', '6.09', '5.65', '5.2', '3.2', '6.65',
       '8(в', '7.12', '5.95', '5.15', '6.35', '2', '6.19', '8.1', '1.4',
       '6.01', '4.93', '3', '2.3', '5.93', '5.72'], dtype=object)

In [19]:
cop = result[result.screen_size == '8(в']
cop.screen_size = None
result[result.screen_size == '8(в'] = cop

/Users/artempozdniakov/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
result.screen_size.unique()

array(['6.1', '5.81', '6.67', '6.7', '6.53', '6.4', '5.8', '6.2', '4.7',
       '6.24', '6.5', '5.4', '6.49', '6.22', '6.26', '6.3', '6', '6.8',
       '5.7', '6.78', '6.81', '6.55', '6.9', '6.59', '6.39', '5.5',
       '5.99', '6.6', '6.44', '7.6', '5', '6.47', '6.57', '2.4', '6.58',
       '5.3', '6.08', '5.71', '6.41', None, '6.15', '5.45', '6.43',
       '1.77', '6.21', '6.56', '2.8', '6.52', '3.5', '6.82', '6.18',
       '5.9', '6.51', '2.2', '5.84', '4.5', '4', '5.6', '6.34', '5.1',
       '1.44', '1.8', '7.3', '6.88', '6.09', '5.65', '5.2', '3.2', '6.65',
       '7.12', '5.95', '5.15', '6.35', '2', '6.19', '8.1', '1.4', '6.01',
       '4.93', '3', '2.3', '5.93', '5.72'], dtype=object)

In [21]:
result.screen_size = result.screen_size.astype('float')

In [25]:
result.popularity+= 1

,best_price,salers_amount,brand_name,model_name,lowest_price,highest_price,stats,screen_size,memory_size,battery_size,release_date,system,popularity
0,33992.0,101,Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0,"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...",6.10,128 ГБ •,2815 мАг (незнімний),10.2020,Apple iOS,1
1,12106.0,37,Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0,"Смартфон • 1 SIM • екран: 5,81"" • OLED •...",5.81,128 ГБ •,"Li-Pol, 3140 мАг (нез",8.2020,Android 10,2
2,7152.0,59,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,6.67,128 ГБ •,5160 мАг (незнімний),9.2020,Android 10,3
3,20889.0,93,Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0,"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...",6.10,64 ГБ •,3110 мАг (незнімний),9.2019,iOS • ак,4
4,9620.0,55,Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0,"Смартфон • 2 SIM • екран: 6,7"" • Super A...",6.70,128 ГБ •,7000 мАг (незнімний),9.2020,Android 10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,326.0,14,Viaan,V241 (Black),222.0,373.0,"Мобільний телефон • 2 SIM • екран: 2,4"" • ...",2.40,"0,032 ГБ •",1000 мАг (знімний) •,9.2016,NaN,1271
1220,5600.0,1,myPhone,Hammer Energy Black,NaN,NaN,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...",5.00,16 ГБ •,"Li-Pol, 5000 мАг (нез",1.2017,Android 6.,1272
1221,2999.0,1,Sigma mobile,X-treme PQ24 Black/Orange,NaN,NaN,Смартфон • 2 або 1 + карта пам'яті SIM • е...,5.00,8 ГБ • о,Li-Pol 5000 mAh (незм,1.2017,Android 7.,1273
1222,5628.0,11,AGM,A10 6/128GB Black,5297.0,6584.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,5.70,128 ГБ •,4400 мАг (незнімний),10.2020,Android 9.,1274


In [26]:
result.memory_size.unique()

array([' 128 ГБ •  ', ' 64 ГБ •   ', ' 256 ГБ •  ', ' 32 ГБ •   ',
       ' 16 ГБ •   ', ' 512 ГБ •  ', nan, ' 0,128 ГБ •', ' 0,032 ГБ •',
       ' 0,016 ГБ •', ' 4 ГБ •   о', ' 1000 ГБ • ', ' 8GB ГБ •  ',
       ' 8 ГБ •   о', ' 4 ГБ •   а', ' 0,064 ГБ •', ' 0.004 ГБ •',
       ' 0.0032 ГБ ', ' 16/32 ГБ •', ' 32 GB ГБ •', ' 0,004 ГБ •'],
      dtype=object)

In [29]:
result.memory_size = result.memory_size.str.replace('ГБ', '')
result.memory_size = result.memory_size.str.replace('GB', '')
result.memory_size = result.memory_size.str.replace(' ', '')
result.memory_size = result.memory_size.str.replace('•', '')
result.memory_size = result.memory_size.str.replace('а', '')
result.memory_size = result.memory_size.str.replace('о', '')
result.memory_size = result.memory_size.str.replace(',', '.')
result.memory_size = result.memory_size.str.replace('16/', '')

In [30]:
result.memory_size.unique()

array(['128', '64', '256', '32', '16', '512', nan, '0.128', '0.032',
       '0.016', '4', '1000', '8', '0.064', '0.004', '0.0032'],
      dtype=object)

In [31]:
result.memory_size = result.memory_size.astype('float')

In [32]:
result

,best_price,salers_amount,brand_name,model_name,lowest_price,highest_price,stats,screen_size,memory_size,battery_size,release_date,system,popularity
0,33992.0,101,Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0,"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...",6.10,128.000,2815 мАг (незнімний),10.2020,Apple iOS,1
1,12106.0,37,Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0,"Смартфон • 1 SIM • екран: 5,81"" • OLED •...",5.81,128.000,"Li-Pol, 3140 мАг (нез",8.2020,Android 10,2
2,7152.0,59,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,6.67,128.000,5160 мАг (незнімний),9.2020,Android 10,3
3,20889.0,93,Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0,"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...",6.10,64.000,3110 мАг (незнімний),9.2019,iOS • ак,4
4,9620.0,55,Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0,"Смартфон • 2 SIM • екран: 6,7"" • Super A...",6.70,128.000,7000 мАг (незнімний),9.2020,Android 10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,326.0,14,Viaan,V241 (Black),222.0,373.0,"Мобільний телефон • 2 SIM • екран: 2,4"" • ...",2.40,0.032,1000 мАг (знімний) •,9.2016,NaN,1271
1220,5600.0,1,myPhone,Hammer Energy Black,NaN,NaN,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...",5.00,16.000,"Li-Pol, 5000 мАг (нез",1.2017,Android 6.,1272
1221,2999.0,1,Sigma mobile,X-treme PQ24 Black/Orange,NaN,NaN,Смартфон • 2 або 1 + карта пам'яті SIM • е...,5.00,8.000,Li-Pol 5000 mAh (незм,1.2017,Android 7.,1273
1222,5628.0,11,AGM,A10 6/128GB Black,5297.0,6584.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,5.70,128.000,4400 мАг (незнімний),10.2020,Android 9.,1274


In [57]:
result.battery_size = battery_size

In [58]:
result.battery_size.unique()

array([' 2815 мАг (незнімний) ', ' Li-Pol, 3140 мАг (нез',
       ' 5160 мАг (незнімний) ', ' 3110 мАг (незнімний) ',
       ' 7000 мАг (незнімний) ', ' 5020 мАг (незнімний) ',
       ' 2942 мАг (незнімний) ', ' 6000 мАг (незнімна) •',
       ' 3687 мАг (незнімний) ', ' 6000 мАг (незнімний) ',
       ' 2716 мАг (незнімний) ', ' 4000 мАг (незнімна) •',
       ' 4500 мАг (незнімний) ', ' 1828 мАг (незнімний) ',
       ' 3190 мАг (незнімний) ', ' 3885 мАг (незнімна) •',
       ' 3400 (незнімний) •   ', ' 4500 мАг (незнімна) •',
       ' Li-Ion Polymer, 5020 ', ' 2227 мАг (незнімний) ',
       ' 4300 мАг •   камера: ', ' Li-ion, 4500 мАг (нез',
       ' 4000 мАг (незнімний) ', ' 5000 мАч (незнімний) ',
       ' 5000 мАч (незнімна) •', ' 2568 мАг (незнімний) ',
       ' 4000 (незнімний) •   ', ' Li-ion, 3000 мАг (нез',
       ' 3969 мАг (незнімний) ', ' 4000 mAh, Li-Po •   к',
       ' 3750 мАг (незмінний) ', ' 3500 мАг (незнімний) ',
       ' 3500 (незнімний) •   ', ' Li-ion, 4000 мАг (нез

In [68]:
new_values = []
for i in result.battery_size:
    if i == None:
        new_values.append(None)
    elif ' м' in i:
        new_values.append(i.split(' м')[0])
    elif ' •' in i:
        new_values.append(i.split(' •')[0])
    elif ' m' in i:
        new_values.append(i.split(' m')[0])
    elif ' (н' in i:
        new_values.append(i.split(' (н')[0])
    elif 'літій-полімерна ' in i:
        new_values.append(i.split('літій-полімерна ')[1])
    elif ' (з' in i:
        new_values.append(i.split(' (з')[0])
    elif 'Lithium-ion polymer' in i:
        new_values.append(None)
    elif 'Polymer electric core' in i:
        new_values.append(None)
    else: 
        new_values.append(i)

In [70]:
result.battery_size = new_values

In [89]:
result.battery_size.unique()

array(['2815', '3140', '5160', '3110', '7000', '5020', '2942', '6000',
       '3687', '2716', '4000', '4500', '1828', '3190', '3885', '3400',
       '2227', '4300', '5000', '2568', '3000', '3969', '3750', '3500',
       '1960', '4080', None, '4510', '4700', '3100', '3600', '4820',
       '3174', '4100', '4025', '2900', '4115', '4720', '4015', '4200',
       '4160', '10000', '1500', '3430', '2915', '2800', '3800', '3010',
       '3520', '5080', '3300', '3330', '4230', '4400', '3700', '800',
       '4680', '6600', '4800', '3340', '3650', '1450', '3550', '5180',
       '4315', '3200', '2510', '8000', '3020', '4050', '5150', '13000',
       '1750', '2870', '1150', '1900', '4650', '1715', '1020', '4380',
       '3505', '6500', '1000', '2700', '1200', '1624', '6580', '5580',
       '4350', '5500', '6300', '2550', '600', '2000', '2730', '5400',
       '2400', '2600', '1570', '2500', '6200', '3765', '1600', '9000',
       '1810', '3080', '15800', '4030', '1400', '2650', '3760', '1510',
       

In [88]:
result.battery_size = result.battery_size.str.replace('Li-Pol', '')
result.battery_size = result.battery_size.str.replace('Li-Ion', '')
result.battery_size = result.battery_size.str.replace(' подвійна ', '')
result.battery_size = result.battery_size.str.replace('(незнімний)', '')
result.battery_size = result.battery_size.str.replace('Lithium-Ion', '')
result.battery_size = result.battery_size.str.replace('Li-Ion Polymer', '')
result.battery_size = result.battery_size.str.replace('Li-Po', '')
result.battery_size = result.battery_size.str.replace('Li-lon', '')
result.battery_size = result.battery_size.str.replace('(знімний)', '')
result.battery_size = result.battery_size.str.replace('Polymer', '')
result.battery_size = result.battery_size.str.replace('Li-ion', '')
result.battery_size = result.battery_size.str.replace('(незмінний)', '')
result.battery_size = result.battery_size.str.replace('(незнімна)', '')
result.battery_size = result.battery_size.str.replace('Літій-іонний', '')
result.battery_size = result.battery_size.str.replace('Li-polymer', '')
result.battery_size = result.battery_size.str.replace('Li-ion', '')
result.battery_size = result.battery_size.str.replace('(', '')
result.battery_size = result.battery_size.str.replace(')', '')
result.battery_size = result.battery_size.str.replace('mAh', '')
result.battery_size = result.battery_size.str.replace('мАч', '')
result.battery_size = result.battery_size.str.replace('+', '')
result.battery_size = result.battery_size.str.replace('мАгод', '')
result.battery_size = result.battery_size.str.replace(',', '')
result.battery_size = result.battery_size.str.replace('Li-pol', '')
result.battery_size = result.battery_size.str.replace('МАг', '')
result.battery_size = result.battery_size.str.replace(' ', '')



In [91]:
result.battery_size = result.battery_size.astype('float')

In [93]:
result.popularity += 1
result

,best_price,salers_amount,brand_name,model_name,lowest_price,highest_price,stats,screen_size,memory_size,battery_size,release_date,system,popularity
0,33992.0,101,Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0,"Смартфон • 2 SIM • екран: 6,1"" • OLED • ...",6.10,128.000,2815.0,10.2020,Apple iOS,1
1,12106.0,37,Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0,"Смартфон • 1 SIM • екран: 5,81"" • OLED •...",5.81,128.000,3140.0,8.2020,Android 10,2
2,7152.0,59,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,6.67,128.000,5160.0,9.2020,Android 10,3
3,20889.0,93,Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0,"Смартфон • 2 SIM • екран: 6,1"" • IPS • ...",6.10,64.000,3110.0,9.2019,iOS • ак,4
4,9620.0,55,Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0,"Смартфон • 2 SIM • екран: 6,7"" • Super A...",6.70,128.000,7000.0,9.2020,Android 10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,326.0,14,Viaan,V241 (Black),222.0,373.0,"Мобільний телефон • 2 SIM • екран: 2,4"" • ...",2.40,0.032,1000.0,9.2016,NaN,1220
1220,5600.0,1,myPhone,Hammer Energy Black,NaN,NaN,"Смартфон • 2 SIM • екран: 5"" • IPS • 1...",5.00,16.000,5000.0,1.2017,Android 6.,1221
1221,2999.0,1,Sigma mobile,X-treme PQ24 Black/Orange,NaN,NaN,Смартфон • 2 або 1 + карта пам'яті SIM • е...,5.00,8.000,5000.0,1.2017,Android 7.,1222
1222,5628.0,11,AGM,A10 6/128GB Black,5297.0,6584.0,Смартфон • 2 або 1 + карта пам'яті SIM • е...,5.70,128.000,4400.0,10.2020,Android 9.,1223


In [103]:
result.system.unique()

array(['iOS', 'Android', nan, '***', 'OxygenOS', 'KAIOS', 'EMUI',
       'WindowsPhone'], dtype=object)

In [102]:
result.system = result.system.str.replace('Apple', '')
result.system = result.system.str.replace('8.1 Oreo', 'Android')
result.system = result.system.str.replace('н', '')
result.system = result.system.str.replace('10', '')
result.system = result.system.str.replace('8', '')
result.system = result.system.str.replace('5.', '')
result.system = result.system.str.replace('Propiereta','***')
result.system = result.system.str.replace('•', '')
result.system = result.system.str.replace('Ka', '')
result.system = result.system.str.replace('6', '')
result.system = result.system.str.replace('™', '')
result.system = result.system.str.replace('Windows Ph', 'Windows Phone')
result.system = result.system.str.replace('9', '')
result.system = result.system.str.replace('9.', '')
result.system = result.system.str.replace('6.', '')
result.system = result.system.str.replace('7', '')
result.system = result.system.str.replace('•   ак', '')
result.system = result.system.str.replace('•   ка', '')
result.system = result.system.str.replace('7.', '')
result.system = result.system.str.replace('N', '')
result.system = result.system.str.replace('•   а', '')
result.system = result.system.str.replace('11', '')
result.system = result.system.str.replace('8.', '')
result.system = result.system.str.replace('Пропрієтар', '***')
result.system = result.system.str.replace('пропріетар', '***')
result.system = result.system.str.replace('6', '')
result.system = result.system.str.replace('4.', '')
result.system = result.system.str.replace('6.', '')
result.system = result.system.str.replace(' ', '')
result.system = result.system.str.replace('.', '')
result.system = result.system.str.replace('а', '')
result.system = result.system.str.replace('к', '')
result.system = result.system.str.replace('+', '')
result.system = result.system.str.replace('Andriod', 'Android')
result.system = result.system.str.replace(' ', '')

In [104]:
cop = result[result.system == '***']
cop.system = None
result[result.system == '***'] = cop

/Users/artempozdniakov/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [106]:
result = result.drop(columns='stats')

In [107]:
result.columns

Index(['best_price', 'salers_amount', 'brand_name', 'model_name',
       'lowest_price', 'highest_price', 'screen_size', 'memory_size',
       'battery_size', 'release_date', 'system', 'popularity'],
      dtype='object')

In [108]:
result.rename(columns={'system':'os', 'memory_size':'memory'})

,best_price,salers_amount,brand_name,model_name,lowest_price,highest_price,screen_size,memory,battery_size,release_date,os,popularity
0,33992.0,101,Apple,iPhone 12 Pro 128GB Pacific Blue (MGMN3/MGLR3),31849.0,42999.0,6.10,128.000,2815.0,10.2020,iOS,1
1,12106.0,37,Google,Pixel 4a 6/128GB Just Black,11740.0,12999.0,5.81,128.000,3140.0,8.2020,Android,2
2,7152.0,59,Xiaomi,Poco X3 NFC 6/128GB Cobalt Blue,6836.0,7999.0,6.67,128.000,5160.0,9.2020,Android,3
3,20889.0,93,Apple,iPhone 11 64GB Black (MWLT2),18200.0,26499.0,6.10,64.000,3110.0,9.2019,iOS,4
4,9620.0,55,Samsung,Galaxy M51 6/128GB Black (SM-M515FZKD),8555.0,10394.0,6.70,128.000,7000.0,9.2020,Android,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1219,326.0,14,Viaan,V241 (Black),222.0,373.0,2.40,0.032,1000.0,9.2016,NaN,1220
1220,5600.0,1,myPhone,Hammer Energy Black,NaN,NaN,5.00,16.000,5000.0,1.2017,Android,1221
1221,2999.0,1,Sigma mobile,X-treme PQ24 Black/Orange,NaN,NaN,5.00,8.000,5000.0,1.2017,Android,1222
1222,5628.0,11,AGM,A10 6/128GB Black,5297.0,6584.0,5.70,128.000,4400.0,10.2020,Android,1223


In [114]:
cols=['brand_name', 'model_name', 'os' , 'popularity', 'best_price', 'lowest_price', 
 'highest_price', 'salers_amount', 'screen_size', 'memory_size', 'battery_size', 'release_date']
result = result[cols]

In [117]:
result = result.sort_values('model_name')

In [119]:
result.index = np.arange(1224)

In [135]:
result.release_date = str(result.release_date.values)
result.release_date = result.release_date.str.replace('.', '-')

In [136]:
result.release_date.unique()

array(["['0       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n1       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n2       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n3       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n4       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n                              ---                        \\n1219    0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n1220    0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n1221    0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n1222    0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n1223    0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\nName: release_date, Length: 1224, dtype: object'\n '0       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n1       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n2       0       10-2020\\\\n1        9-2020\\\\n2        9-2---\\n3       0       10-2

In [125]:
result = result.rename(columns={'salers_amount':'sellers_amount'})

In [126]:
result.to_csv('ds.csv')

In [153]:
result = result.drop(columns='Unnamed: 0')

In [154]:
result

,brand_name,model_name,os,popularity,best_price,lowest_price,highest_price,sellers_amount,screen_size,memory_size,battery_size,release_date
0,ALCATEL,1 1/8GB Bluish Black (5033D-2JALUAA),Android,803,1690.0,1529.0,1819.0,36,5.00,8.0,2000.0,10.2020
1,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,902,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9.2020
2,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,926,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9.2020
3,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,938,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9.2020
4,Nokia,1.3 1/16GB Charcoal,Android,178,1999.0,NaN,NaN,10,5.71,16.0,3000.0,4.2020
...,...,...,...,...,...,...,...,...,...,...,...,...
1219,Apple,iPhone XS Max 64GB Gold (MT522),iOS,124,22685.0,16018.0,27900.0,61,6.50,64.0,3174.0,9.2018
1220,Apple,iPhone XS Max Dual Sim 64GB Gold (MT732),iOS,695,24600.0,21939.0,33720.0,28,6.50,64.0,3174.0,9.2018
1221,HUAWEI,nova 5T 6/128GB Black (51094MEU),Android,51,8804.0,7999.0,9999.0,18,6.26,128.0,3750.0,11.2019
1222,ZTE,nubia Red Magic 5G 8/128GB Black,Android,473,18755.0,18500.0,19010.0,2,6.65,128.0,4500.0,10.2020


In [155]:
new_dates = []
for i in result.release_date:
    new_dates.append(str(i))

In [158]:
nn_dates = []
for i in new_dates:
    if i[-4] == '.':
        nn_dates.append(i+'0')
    elif len(i) > 7:
        nn_dates.append(i.split('000')[0]+'0')
    else:
        nn_dates.append(i)

In [161]:
result.release_date = nn_dates

In [162]:
result.release_date = result.release_date.str.replace('.', '-')

In [163]:
result

,brand_name,model_name,os,popularity,best_price,lowest_price,highest_price,sellers_amount,screen_size,memory_size,battery_size,release_date
0,ALCATEL,1 1/8GB Bluish Black (5033D-2JALUAA),Android,803,1690.0,1529.0,1819.0,36,5.00,8.0,2000.0,10-2020
1,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,902,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9-2020
2,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,926,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9-2020
3,ALCATEL,1 5033D 1/16GB Volcano Black (5033D-2LALUAF),Android,938,1803.0,1659.0,2489.0,36,5.00,16.0,2000.0,9-2020
4,Nokia,1.3 1/16GB Charcoal,Android,178,1999.0,NaN,NaN,10,5.71,16.0,3000.0,4-2020
...,...,...,...,...,...,...,...,...,...,...,...,...
1219,Apple,iPhone XS Max 64GB Gold (MT522),iOS,124,22685.0,16018.0,27900.0,61,6.50,64.0,3174.0,9-2018
1220,Apple,iPhone XS Max Dual Sim 64GB Gold (MT732),iOS,695,24600.0,21939.0,33720.0,28,6.50,64.0,3174.0,9-2018
1221,HUAWEI,nova 5T 6/128GB Black (51094MEU),Android,51,8804.0,7999.0,9999.0,18,6.26,128.0,3750.0,11-2019
1222,ZTE,nubia Red Magic 5G 8/128GB Black,Android,473,18755.0,18500.0,19010.0,2,6.65,128.0,4500.0,10-2020


In [164]:
result.to_csv('ds2.csv')